# Install & Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import torch
import os
import shutil
import random

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from glob import glob

import yaml

In [ ]:
!pip install wandb

'''
이거 깃헙 올릴 때는 삭제하셈!!!!!!!!!!!
my API key :

a7ddd85e3da80c4a8503ee85e4685caa32deb381
'''

# YOLO V5 설치

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install



# Custom Dataset 정리

In [ ]:

## Unzip Image File
%cd /content
!unzip -qq "/content/gdrive/MyDrive/Colab Notebooks/YOLO V5/cross.zip" ## Folder 6개가 생김
print('Complete!')

In [ ]:
'''
압축이 해제된 폴더들은 Object의 수에 따라 1~6으로 구분 해둔 것임
따라서 일단 안의 파일들을 다 하나로 합쳐준 다음 Train Val 분리를 해주기로 결정
'''

os.makedirs('./dataset') ## 1~6으로 분할 되어있는 파일을 다 모아줄 폴더 생성

to_here = '/content/dataset'


for i in tqdm(range(1,7)) :
    get_file = os.listdir(f'/content/교차로정보 데이터셋_bbox_{i}') ## 폴더 내의 파일들 리스트로 묶음
    for j in get_file :
        shutil.move(f'/content/교차로정보 데이터셋_bbox_{i}/'+j, to_here) ## dataset 폴더로 모두 이동
    os.rmdir(f'/content/교차로정보 데이터셋_bbox_{i}') 

In [ ]:
## YOLO V5 폴더의 data 파일에 image - train/valid 폴더 생성
os.makedirs('/content/yolov5/data/images/train') 
os.makedirs('/content/yolov5/data/images/valid')

## 라벨도 형성
os.makedirs('/content/yolov5/data/labels')
os.makedirs('/content/yolov5/data/labels/train')
os.makedirs('/content/yolov5/data/labels/valid') 

img_lst = []
label_lst = []


get_file = os.listdir(f'/content/dataset') ## 폴더 내의 파일들 리스트로 묶음
for j in get_file :
    if '.jpg' in j :
        img_lst.append(j)
    elif 'json' in j :
        label_lst.append(j)

train_img , val_img = train_test_split(img_lst, test_size=0.2, random_state=42) ## train set과 validation set 분리

train_lab = []
val_lab = []
for i in tqdm(label_lst) :
    if i.split('.')[0] + '.jpg' in train_img :
        train_lab.append(i)

    elif i.split('.')[0] + '.jpg' in val_img: 
        val_lab.append(i)

In [ ]:
print(f'Train Image 수 : {len(train_img)}')
print(f'Train Image 수 : {len(train_lab)}')
print(f'Train Image 수 : {len(val_img)}')
print(f'Train Image 수 : {len(val_lab)}')

In [ ]:
for a, b in zip(train_img, train_lab) :
    shutil.move(f'/content/dataset/{a}', '/content/yolov5/data/images/train')
    shutil.move(f'/content/dataset/{b}', '/content/yolov5/data/images/valid')

for c,d in zip(val_img, val_lab) : 
    shutil.move(f'/content/dataset/{c}', '/content/yolov5/data/labels/train')
    shutil.move(f'/content/dataset/{d}', '/content/yolov5/data/labels/valid')



In [ ]:
train_path = list(glob('/content/yolov5/data/images/train/*.jpg'))
valid_path = list(glob('/content/yolov5/data/images/valid/*.jpg'))

In [ ]:
## YOLO V5 는 파일의 경로를 txt로 알려줘야 한다
with open('/content/yolov5/data/images/train.txt','w') as f :
    f.write('\n'.join(train_path) + '\n')

with open('/content/yolov5/data/images/valid.txt','w') as f :
    f.write('\n'.join(valid_path) + '\n')

In [ ]:
## Custom Dataset 용 yaml 파일 생성
with open('/content/yolov5/data/dataset.yaml', 'w') as f:

    f.close()


with open('/content/yolov5/data/dataset.yaml', 'r') as f:
    data = yaml.safe_load(f)

## 학습 데이터, 검증 데이터 경로 지정
data['train'] = '/content/yolov5/data/images/train'
data['valid'] = '/content/yolov5/data/images/valid'

## 클래스 수
data['nc'] = 3

## 종류
data ['names'] = ['Zebra_Cross', 'G_Signal', 'R_Signal']

In [ ]:
%cd /content/yolov5

In [ ]:
!pip install -U PyYAML

In [ ]:
!python train.py --img 640 --batch 32 --epochs 50 --data ./data/dataset.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolo_test

In [ ]:
https://colab.research.google.com/drive/1IHpT1Cv17bJVleWBEyRifGJiPyFFVv0o?usp=sharing#scrollTo=TQQ-ZJApwKCZ